In [1]:
# Key challenge: current kernels do not reflect medicinal chemistry thinking wrt similarity
from rdkit import Chem
from rdkit.Chem import FragmentMatcher
from rdkit.Chem import AllChem
from rdkit.Chem import rdFingerprintGenerator
import numpy as np
from rdkit.Chem import DataStructs

In [2]:


def create_hybrid_fingerprint(mol, radius=2, nBits=2048):
    """
    Creates a hybrid fingerprint that combines Morgan count fingerprint with pharmacophore features.
    Currently only has one bit to denote carboxylic acid or amide groups

    Args:
        mol: RDKit molecule object
        radius: Radius for Morgan fingerprint
        nBits: Number of bits for Morgan fingerprint

    Returns:
        Numpy array with Morgan fingerprint counts + 1 extra count for acid/amide
    """
    # Generate Morgan count fingerprint
    morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
    morgan_fp = morgan_gen.GetCountFingerprint(mol)
    fp_array = np.zeros((nBits,))
    DataStructs.ConvertToNumpyArray(morgan_fp, fp_array)

    # Initialize matchers for acid and amide
    acid_matcher = FragmentMatcher.FragmentMatcher()
    acid_matcher.Init('C(=O)[OH]')
    amide_matcher = FragmentMatcher.FragmentMatcher()
    amide_matcher.Init('C(=O)N')  # Basic amide pattern


    # SMARTS pattern for carboxylic acid and amide

    # Count acid and amide occurrences
    acid_count = len(acid_matcher.GetMatches(mol))
    amide_count = len(amide_matcher.GetMatches(mol))

    # Combine fingerprints with total acid/amide count
    hybrid_fp = np.append(fp_array, acid_count + amide_count)
    return hybrid_fp  

In [12]:
from sklearn.gaussian_process.kernels import Kernel, GenericKernelMixin
from sklearn.gaussian_process.kernels import Hyperparameter

class TanimotoKernel(GenericKernelMixin, Kernel):
    """
    Implementation of a weighted hybrid Tanimoto kernel that combines ECFP fingerprint similarity
    with pharmacophore feature similarity. The contribution of the pharmacophore features to the
    overall similarity can be controlled through a weight parameter.

    The kernel is defined as:
    k(x,y) = (1-w) * k_ecfp(x,y) + w * k_pharm(x,y)

    where:
    - w is the weight for pharmacophore features (between 0 and 1)
    - k_ecfp is the Tanimoto similarity for ECFP fingerprints
    - k_pharm is the Tanimoto similarity for pharmacophore features
    """

    def __init__(self, pharm_weight=0.5):
        """Initialize the kernel with a weight parameter for pharmacophore features.

        Args:
            pharm_weight (float): Weight for pharmacophore features (between 0 and 1).
                                 Default is 0.5 for equal weighting.
        """
        self.pharm_weight = pharm_weight

    @property
    def hyperparameters(self):
        """Returns a list of all hyperparameter specifications."""
        return [Hyperparameter("pharm_weight", "numeric", bounds=(0, 1))]

    def _tanimoto_similarity(self, x, y):
        """Compute Tanimoto similarity between two vectors."""
        # Reshape x and y to 2D if they are 1D
        x = x.reshape(1, -1) if x.ndim == 1 else x
        y = y.reshape(1, -1) if y.ndim == 1 else y

        xy = np.dot(x, y.T)
        xx = np.sum(x * x, axis=1)[:, np.newaxis]
        yy = np.sum(y * y, axis=1)
        return xy / (xx + yy - xy)


    def __call__(self, X, Y=None, eval_gradient=False):
        """
        Compute the weighted hybrid Tanimoto kernel between X and Y.

        Args:
            X (array-like): First set of fingerprints of shape (n_samples_X, n_features)
            Y (array-like, optional): Second set of fingerprints of shape (n_samples_Y, n_features)
                If None, use X as Y.
            eval_gradient (bool): Whether to evaluate the gradient. Default is False.

        Returns:
            array-like: Kernel matrix of shape (n_samples_X, n_samples_Y)
            If eval_gradient is True, returns a tuple (kernel_matrix, kernel_gradient)
        """
        X = np.asarray(X)
        if Y is None:
            Y = X
        else:
            Y = np.asarray(Y)

        # Split fingerprints into ECFP and pharmacophore parts
        # Assuming the last column is the pharmacophore features
        X_ecfp = X[:, :-1]
        X_pharm = X[:, -1:]
        Y_ecfp = Y[:, :-1]
        Y_pharm = Y[:, -1:]

        # Compute similarities for both parts
        K_ecfp = self._tanimoto_similarity(X_ecfp, Y_ecfp)
        K_pharm = self._tanimoto_similarity(X_pharm, Y_pharm)

        # Combine with weights
        K = (1 - self.pharm_weight) * K_ecfp + self.pharm_weight * K_pharm

        if eval_gradient:
            # Compute gradient with respect to pharm_weight
            gradient = K_pharm - K_ecfp
            if Y is X:
                return K, gradient[:, :, np.newaxis]
            else:
                return K, gradient[:, :, np.newaxis]
        return K

    def diag(self, X):
        """Returns the diagonal of the kernel k(X, X)."""
        return np.ones(X.shape[0])

    def is_stationary(self):
        """Returns whether the kernel is stationary."""
        return False

    def get_params(self, deep=True):
        """Get parameters of this kernel."""
        return {"pharm_weight": self.pharm_weight}

    def set_params(self, **params):
        """Set the parameters of this kernel."""
        if "pharm_weight" in params:
            self.pharm_weight = params["pharm_weight"]
        return self

    @property
    def theta(self):
        """Returns the parameters of the kernel."""
        return np.array([self.pharm_weight])

    @theta.setter
    def theta(self, theta):
        """Sets the parameters of the kernel."""
        self.pharm_weight = theta[0]

In [6]:
aspirin = Chem.MolFromSmiles('CC(=O)OC1=CC=CC=C1C(=O)O')  # Has acid group
amide_aspirin = Chem.MolFromSmiles('CC(=O)Oc1ccccc1C(=O)N')  # Has amide group

# Generate fingerprints
fps = []
for mol in [aspirin, amide_aspirin]:
    fp = create_hybrid_fingerprint(mol)
    fps.append(fp)
X = np.array(fps)

# Test kernel with different weights for pharmacophore features
weights = [0.0, 0.5, 1.0]

print("Testing kernel with different pharmacophore weights:")
print("-" * 50)

for w in weights:
    kernel = TanimotoKernel(pharm_weight=w)
    K = kernel(X)

    print(f"\n=== Pharmacophore weight: {w} ===")
    print("Kernel matrix:")
    print(K)

Testing kernel with different pharmacophore weights:
--------------------------------------------------

=== Pharmacophore weight: 0.0 ===
Kernel matrix:
[[1.         0.85294118]
 [0.85294118 1.        ]]

=== Pharmacophore weight: 0.5 ===
Kernel matrix:
[[1.         0.92647059]
 [0.92647059 1.        ]]

=== Pharmacophore weight: 1.0 ===
Kernel matrix:
[[1. 1.]
 [1. 1.]]



### Option 2: pharmacophore transforms through additive fingerprints

In [8]:
from rdkit.Chem import AllChem
bioisosteres = [AllChem.ReactionFromSmarts('[C:1](=[O:2])[OH:3]>>[C:1](=[O:2])[NH2:3]')]
     

ref_set = [Chem.MolFromSmiles('OC(=O)C1CCCCC1'),
           Chem.MolFromSmiles('OC(=O)C1CCCCN1'),
           Chem.MolFromSmiles('OC(=O)C1=CC=CC=C1'),
           Chem.MolFromSmiles('OC(=O)C1=NC=CC=C1'),
           Chem.MolFromSmiles('OC(=O)C1=NC=CC=N1')
           ]
ref_set_bioisosteres = [bioisosteres[0].RunReactants((mol,))[0][0] for mol in ref_set]
for i in ref_set_bioisosteres:
  Chem.SanitizeMol(i)
    

In [9]:

morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
diffs = []
for i,j in zip(ref_set,ref_set_bioisosteres):
  morgan_fp = morgan_gen.GetCountFingerprint(i)
  morgan_fp_biois = morgan_gen.GetCountFingerprint(j)
  fp_array = np.zeros((2048,))
  fp_array_biois = np.zeros((2048,))
  DataStructs.ConvertToNumpyArray(morgan_fp, fp_array)
  DataStructs.ConvertToNumpyArray(morgan_fp_biois, fp_array_biois)
  diffs.append(fp_array-fp_array_biois)

diffs = np.array(diffs)
diffs.shape



(5, 2048)

In [7]:

aspirin = Chem.MolFromSmiles('CC(=O)OC1=CC=CC=C1C(=O)O')  # Has acid group
amide_aspirin = Chem.MolFromSmiles('CC(=O)Oc1ccccc1C(=O)N')  # Has amide group

       

In [10]:
amide_aspirin_fp = morgan_gen.GetCountFingerprint(amide_aspirin)
amide_aspirin_fp_np = np.zeros((2048,))
DataStructs.ConvertToNumpyArray(amide_aspirin_fp, amide_aspirin_fp_np)
     

transformed = amide_aspirin_fp_np+diffs
     

aspirin_fp = morgan_gen.GetCountFingerprint(aspirin)
aspirin_fp_np = np.zeros((2048,))
DataStructs.ConvertToNumpyArray(aspirin_fp, aspirin_fp_np)
     

for i in range(transformed.shape[0]):
  print(TanimotoKernel()._tanimoto_similarity(transformed[i],aspirin_fp_np))

[[0.83783784]]
[[0.83783784]]
[[0.88732394]]
[[0.88732394]]
[[0.88732394]]


In [11]:

# sample ECFP implementation
def generateAtomInvariant(mol):
    """
    >>> generateAtomInvariant(Chem.MolFromSmiles("Cc1ncccc1"))
    [341294046, 3184205312, 522345510, 1545984525, 1545984525, 1545984525, 1545984525]
    """
    num_atoms = mol.GetNumAtoms()
    invariants = [0]*num_atoms
    for i,a in enumerate(mol.GetAtoms()):
        descriptors=[]
        descriptors.append(a.GetAtomicNum())
        descriptors.append(a.GetTotalDegree())
        descriptors.append(a.GetTotalNumHs())
        descriptors.append(a.IsInRing())
        descriptors.append(a.GetIsAromatic())
        invariants[i]=hash(tuple(descriptors))& 0xffffffff
    return invariants
generateAtomInvariant(Chem.MolFromSmiles("Cc1ncccc1"))
     

[1706026559,
 3752853591,
 864367768,
 3108496040,
 3108496040,
 3108496040,
 3108496040]